In [1]:
import os
import sys
sys.path.append("..")

import tensorflow as tf

import cst_model as cst
import distortion_layers as ly
import base_models as bm
import callbacks as cb

# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) 

In [2]:
os.getcwd()

'/main_dir/felipe/projects/CST/notebooks'

### Parameters


In [3]:
# train_path = "../data"
# train_path = "/main_dir/felipe/projects/cp_toolbox_data/artifacts_project/training_data/v4"
train_path = "/main_dir/felipe/projects/cp_toolbox_data/tiles"
model_path = "../models"
model_base_name = "test_model"


tile_size = 128
batch_size = 32
channels = 3
n_st_components = 2
alpha = 2
epochs = 10
class_mode = "sparse"

validation_split = 0.2

### Load dataset

In [4]:
# TODO

### Create distortion layer

In [5]:
dcs = {
    "contrast": {"factor": [0.2, 0.2]},
    "color": {"factor": [20,0,20]},
    "blur": {"filter_shape": 2, "sigma": 5.},  # kernel size is 'filter_shape * 2 + 1'
    "brightness": {"lower": .85, "upper":1.15}
}

layers = [
    ly.RandomColorByChannel(**dcs["color"]), 
    tf.keras.layers.RandomContrast(**dcs["contrast"]),
    ly.RandomBrightness(**dcs["brightness"]),
    ly.RandomGaussianBlur(**dcs["blur"]),
    ly.BlueRedChannelSwapLayer(),
]


# dist_layer = [AdjSaturation(10),
#               tf.keras.layers.Lambda(lambda x: x * 0. + 55)]

dist_layer = layers



2023-07-06 14:23:14.961277: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-06 14:23:14.977648: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-06 14:23:14.977826: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-06 14:23:14.978441: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Load images and assign parameters

In [6]:
gen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = validation_split
)

t_flow = gen.flow_from_directory(
    directory=train_path,
    target_size=(tile_size,tile_size),
    color_mode='rgb',  # rgb for color
    batch_size=batch_size,
    class_mode=class_mode,  # 'sparse' for multiclass, 'binary' for binary 
    subset='training'
)

v_flow = gen.flow_from_directory(
    directory=train_path,
    target_size=(tile_size,tile_size),
    color_mode="rgb",  # rgb for color
    batch_size=batch_size,
    shuffle=False,
    class_mode=class_mode,  # 'sparse' for multiclass, 'binary' for binary
    subset='validation'
)

Found 124 images belonging to 3 classes.
Found 29 images belonging to 3 classes.


### Load and compile network

In [11]:
""" Callbacks """
callbacks = [
    cb.EpochSaver(
        layer_name=model_base_name, 
        model_path=os.path.join(model_path, model_base_name), 
        base_name=model_base_name
    ),
    tf.keras.callbacks.CSVLogger(
        os.path.join(model_path, model_base_name, model_base_name + ".csv"), 
        ","
    )
]



""" Metrics and loss function """
if class_mode == "binary":
    metrics = [
        tf.keras.metrics.BinaryCrossentropy(name="bce"),  # BinaryCrossentropy for binary
        tf.keras.metrics.BinaryAccuracy(name="acc")
    ]
    base_loss = tf.keras.losses.binary_crossentropy
    final_layer_node = 1
    binary = True
    
elif class_mode == "sparse":
    metrics = [
        tf.keras.metrics.SparseCategoricalCrossentropy(name="sce"), 
        tf.keras.metrics.SparseCategoricalAccuracy(name="acc")
    ]
    base_loss = tf.keras.losses.sparse_categorical_crossentropy
    final_layer_node = t_flow.num_classes
    binary = False

elif class_mode == "categorical":
    metrics = [
        tf.keras.metrics.CategoricalCrossentropy(name="cce"),
        tf.keras.metrics.CategoricalAccuracy(name="acc")
    ]
    base_loss = tf.keras.losses.categorical_crossentropy
    final_layer_node = t_flow.num_classes
    binary = False
    
else:
    print("no class mode provided")
    
    
base_model = bm.create_thesis_model(tile_size=tile_size, channels=3, final_layer_node=final_layer_node)
base_model._name = model_base_name  # name of the layer (i.e. model) we want to save later.

i = tf.keras.Input(shape=(tile_size, tile_size, channels))
x_i = base_model(i)
cst_model = cst.CSTModel(inputs=i, outputs=x_i, alpha=alpha, n_st_components=n_st_components, 
                         dist_layers=dist_layer, binary=binary)


cst_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, amsgrad=True),
    loss = base_loss,
    metrics = metrics,
)


### Train network

In [12]:
# cst_model.run_eagerly = False

cst_model.fit(
    x=t_flow,
    validation_data=v_flow,
    epochs=epochs,
    callbacks=callbacks,
    class_weight=bm.get_class_weights(t_flow.classes)
)

Epoch 1/10
4/4 [==============================] - ETA: 0s - loss_0: 1.6427 - cst_metric: 1.6441 - sce_: 1.1441 - acc_: 0.3968 - sce_0: 1.2024 - acc_0: 0.4104 - sce_1: 1.1788 - acc_1: 0.4202 - sce_2: 1.2181 - acc_2: 0.3920 - sce_3: 1.1900 - acc_3: 0.5170 - sce_4: 1.1103 - acc_4: 0.4496WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
class weights saved to path: 
../models/test_model/test_model_e1.h5
4/4 [==============================] - 17s 557ms/step - loss_0: 1.6035 - cst_metric: 1.6052 - sce_: 1.1407 - acc_: 0.3989 - sce_0: 1.1977 - acc_0: 0.4107 - sce_1: 1.1732 - acc_1: 0.4213 - sce_2: 1.2103 - acc_2: 0.3935 - sce_3: 1.1816 - acc_3: 0.5124 - sce_4: 1.1094 - acc_4: 0.4463 - val_sce: 1.1811 - val_acc: 0.4057
Epoch 2/10
4/4 [==============================] - ETA: 0s - loss_0: 1.1394 - cst_metric: 1.1168 - sce_: 1.2241 - acc_: 0.4063 - sce_0: 1.1624 - acc_0: 0.4122 -

In [8]:
"""
Here we only save the base network we used to encode/predict the input.
If we want to save the whole model with all distortion preprocessing and 
rescale layers, we have to make sure that all custom layers can be saved. 
Otherwise you may get an error like the following:

        Layer RandomColorByChannel has arguments ['self', 'factor']
        in `__init__` and therefore must override `get_config()`.

        Example:

        class CustomLayer(keras.layers.Layer):
            def __init__(self, arg1, arg2):
                super().__init__()
                self.arg1 = arg1
                self.arg2 = arg2

            def get_config(self):
                config = super().get_config()
                config.update({
                    "arg1": self.arg1,
                    "arg2": self.arg2,
                })
                return config
           
           
Other error I encountered was the following, apparently due to tf 
bugs depending on the tf version: 

        TypeError: Unable to serialize [20.  0. 20.] to JSON. 
        Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.

"""

cst_model.get_layer(model_base_name).save("../models/whatever.h5")